In [4]:
import torch
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

class DiseasePredictor:
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
           
        self.llm = pipeline(
            "text-generation",
            model="meta-llama/Llama-3.2-1B-Instruct",
            device_map="auto",
            truncation=True,
            pad_token_id=2  
        )

    def get_follow_up_questions(self, initial_symptoms):
        prompt = (
            f"Given the following initial symptoms described by a patient: {initial_symptoms}, "
            f"generate three of the most important follow-up questions a medical professional "
            f"should ask to gather more detailed and relevant information about the patient's condition. "
            f"Ensure that the questions are concise and medically relevant."
        )
        
        try:
            response = self.llm(
                prompt,
                max_length=1000,
                temperature=0.7,
                return_full_text=False
            )[0]['generated_text']
            
            # Extract questions
            questions = []
            for line in response.split('\n'):
                if '?' in line:
                    questions.append(line.strip())
            return questions[:3]  
        except Exception as e:
            return [
                "How long have you had these symptoms?",
                "Is the pain constant or intermittent?",
                "What makes it better or worse?"
            ]

    def predict_with_llm(self, symptoms_info):
        prompt = (
            f"You are a medical expert analyzing a patient's symptoms and history provided below:\n"
            f"Patient information: {symptoms_info}\n\n"
            f"Based on this information, print the disease he is facing:\n"
            f"Ensure the response is structured and easy to understand."
        )
        
        try:
            response = self.llm(
                prompt,
                max_length=2000,
                temperature=0.7,
                return_full_text=False
            )[0]['generated_text']
            
            # Ensure we have a valid response
            if not response.strip() or len(response) < 10:
                raise ValueError("No response generated by the model.")
                
            return response
            
        except Exception as e:
            # Fallback response
            return (
                "The system could not generate an analysis based on the provided symptoms. "
                "Here are general guidelines you may consider:\n"
                "1. Monitor the progression of your symptoms.\n"
                "2. Record any additional symptoms or changes over time.\n"
                "3. Consult a qualified healthcare professional for an accurate diagnosis and personalized treatment plan."
            )

    def get_recommendations(self, symptoms_info):
        prompt = (
            f"You are a healthcare professional tasked with providing recommendations based on the following symptoms "
            f"described by a patient: {symptoms_info}\n"
            f"Generate a list of general recommendations that the patient should follow to manage their condition. "
        )
        
        try:
            response = self.llm(
                prompt,
                max_length=2000,
                temperature=0.5,
                return_full_text=False
            )[0]['generated_text']
            
            if not response.strip(): 
                raise ValueError("No response generated by the model.")
            
            return response
            
        except Exception as e:
            return (
                "The system could not generate specific recommendations. "
                "Here are general guidelines to consider:\n"
                "1. Monitor your symptoms closely. If they worsen, seek medical attention immediately.\n"
                "2. Stay hydrated and rest as much as possible.\n"
                "3. Avoid any activities or substances that might aggravate your condition.\n"
                "4. Consult a healthcare professional for a detailed assessment."
                
            )


def main():
    print("Medical Analysis System Initialized\n")
    predictor = DiseasePredictor()
    
    while True:
        print("\nDescribe your symptoms (or 'quit' to exit):")
        initial_symptoms = input().strip()
        
        if initial_symptoms.lower() == 'quit':
            break
        
        try:
            # Get follow-up questions
            print("\n=== Follow-up Questions ===")
            questions = predictor.get_follow_up_questions(initial_symptoms)
            
            # Gather additional information
            additional_info = []
            for question in questions:
                print(f"\n{question}")
                answer = input().strip()
                additional_info.append(f"{question}: {answer}")
            
            # Combine all information
            all_symptoms = f"{initial_symptoms}. {'. '.join(additional_info)}"
            
            # Get analysis
            print("\n=== Analysis ===")
            analysis = predictor.predict_with_llm(all_symptoms)
            print(analysis)
            
            # Get recommendations
            print("\n=== Recommendations ===")
            recommendations = predictor.get_recommendations(all_symptoms)
            print(recommendations)
            
            print("\nNOTE: This is not a medical diagnosis. Please consult a healthcare professional.")
            
        except Exception as e:
            print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Medical Analysis System Initialized

Using device: cuda

Describe your symptoms (or 'quit' to exit):

=== Follow-up Questions ===

1.  What is the nature of your back pain? Is it acute or chronic?

2.  How long have you been experiencing this back pain?

3.  Have you experienced any recent trauma or injuries that could be related to your back pain?

=== Analysis ===
 Here is the patient's information:
Patient Name: Not Provided
Patient Information: 
1. Nature of Back Pain: Acute
2. Duration of Back Pain: 4 days
3. Recent Trauma or Injuries: No

The patient is experiencing acute back pain and headache, which is a relatively common condition that can be caused by various factors, including muscle strain, muscle spasms, or even a viral infection. The acute nature of the pain and headache suggests that the patient may be experiencing a sudden onset of symptoms.

Based on the patient's symptoms, it is likely that they are facing a condition known as **Musculoskeletal Pain**. This type of pa